<a href="https://colab.research.google.com/github/tsenga2/keio-quant-macro/blob/main/growth_comp_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt90.loc[pwt90['country'].isin(['France', 'Germany', 'Canada', 'Italy', 'Japan', 'United Kingdom', 'United States'])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share

# Order by year
data = data.sort_values('year')

# Group b y isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['Growth Rate'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['dk'] = (grouped_data['k_pc'].diff() * 100)  # Growth rate of capital per capita
data['TFP Growth'] = data['Growth Rate'] - data['a'] * data['dk'] # Growth rate of TFP per capita

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'Growth Rate': 'mean',
                                           'a': 'mean',
                                           'dk': 'mean',
                                           'TFP Growth':'mean'})

# Calculate additional summary statistics
summary['Capital Deepening'] = summary['a'] * summary['dk']
summary['TFP Share'] = summary['TFP Growth'] / summary['Growth Rate']
summary['Capital Share'] = summary['Capital Deepening'] / summary['Growth Rate']

#
summary.drop(['a','dk'], axis = 1, inplace=True)

# Print output
print(summary)

             Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
countrycode                                                          
CAN             0.744709    0.207389           0.537943   0.278483   
DEU             0.689737    0.288159           0.412125   0.417781   
FRA             0.818749    0.289019           0.530840   0.353001   
GBR             1.104174    0.846882           0.260364   0.766983   
ITA             0.043478   -0.574149           0.622656 -13.205667   
JPN             0.909459    0.016058           0.931534   0.017657   
USA             1.589208    0.993836           0.602823   0.625366   

             Capital Share  
countrycode                 
CAN               0.722353  
DEU               0.597510  
FRA               0.648355  
GBR               0.235800  
ITA              14.321333  
JPN               1.024273  
USA               0.379323  
